In [22]:
from lexmachina import LexMachinaAsyncClient
from lexmachina import DistrictCaseQueryRequest
import asyncio
from typing import Optional
from collections import defaultdict

In [2]:
client = LexMachinaAsyncClient(config_file_path='../config/config.ini')

In [29]:
async def plaintiff_centric_info_for_date_range(start: str, end: str):
    case_infos = await cases_for_date_range(start, end)
    case_ids = list(c['districtCaseId'] for c in case_infos)
    print(f'there were {len(case_ids)} cases filed {start} and {end}')
    cases = await asyncio.gather(*[client.get_district_cases(case) for case in case_ids])
    return [get_plaintiffs_entities_and_judges(c) for c in cases]
    
        
async def cases_for_date_range(start: str, end: str):
    # format date in ISO format `YYYY-MM-DD`
    query = (
        DistrictCaseQueryRequest()
        .set_date(start, "filed", "onOrAfter")
        .set_date(end, "filed", "onOrBefore")
    )
    return await client.query_district_case(query=query, options={'pageThrough': False}, page_size=100)

def get_plaintiffs_entities_and_judges(c):
    judges = c['judges']
    lawfirms = c['lawFirms']
    party_firms = defaultdict(set)
    for lf in lawfirms:
        for client in lf['clientPartyIds']:
            party_firms[client].add((lf['name'], lf['lawFirmId']))
    attys = c['attorneys']
    party_attys = defaultdict(set)
    for a in attys:
        print(a.)
        for client in a['clientPartyIds']:
            party_attys[client].add((a['name'], a['attorneyId'], tuple(sorted(a['lawfirmIds']))))
    plaintiffs = [p for p in c['parties'] if p['role'] == 'Defendant']
    return plaintiffs, lawfirms, attys, judges

In [30]:
start_date = '2023-09-20'
end_date = '2023-09-22'
for parties, lfs, attys, judges in await plaintiff_centric_info_for_date_range(start_date, end_date):
    print(parties, lfs, attys, judges)

{'cases': [{'url': 'https://api.lexmachina.com/district-cases/2009323903', 'districtCaseId': 2009323903}, {'url': 'https://api.lexmachina.com/district-cases/2009907584', 'districtCaseId': 2009907584}, {'url': 'https://api.lexmachina.com/district-cases/2009907593', 'districtCaseId': 2009907593}, {'url': 'https://api.lexmachina.com/district-cases/2009907594', 'districtCaseId': 2009907594}, {'url': 'https://api.lexmachina.com/district-cases/2009907595', 'districtCaseId': 2009907595}]}
there were 5 cases filed 2023-09-20 and 2023-09-22


KeyError: 'clientPartyIds'